In [12]:
from facenet_pytorch import InceptionResnetV1, MTCNN
# from mtcnn import MTCNN
from mtcnn.detector import MtcnnDetector
import cv2
from align_faces import warp_and_crop_face, get_reference_facial_points
import numpy as np
from numpy.linalg import norm
import time
import glob
import torch 
from torchvision import transforms
from facenet_pytorch import InceptionResnetV1, fixed_image_standardization
import os
from PIL import Image
import numpy as np

In [16]:
mtcnn = MTCNN()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = InceptionResnetV1(
    classify=False,
    pretrained="casia-webface"
).to(device)

infor = model.eval()

In [24]:
img = cv2.imread("3Nguoi.jpg")
x = mtcnn(img)

c:\Users\ASUS\anaconda3\lib\site-packages\torch\nn\functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


In [29]:
out = model(x.unsqueeze(0))
print(out.shape)

torch.Size([1, 512])
